<a href="https://colab.research.google.com/github/EmmanuelABonsu/CE888/blob/main/src/project/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50

In [16]:
training_dir = "https://github.com/EmmanuelABonsu/CE888/tree/main/src/project/Training"
testing_dir = "https://github.com/EmmanuelABonsu/CE888/tree/main/src/project/Test"
img_width, img_height = 224, 224 

In [17]:
# Load Pre-trained CNN
pre_trained_model = ResNet50(
    weights='imagenet',
    include_top = False,
    input_shape=(img_width, img_height, 3)
)
pre_trained_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [18]:
# Extracting features 
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

# labels = ['Fire', 'No_Fire']
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

In [19]:
def extract_features_with_pretrained_model(directory, data_size):
  features = np.zeros(shape=(data_size, 7, 7, 2048))
  labels = np.zeros(shape=(data_size,2))

  generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='categorical')
  
  i = 0
  for inputs_batch, labels_batch in generator:
      features_batch = pre_trained_model.predict(inputs_batch)
      features[i * batch_size: (i + 1) * batch_size] = features_batch
      labels[i * batch_size: (i + 1) * batch_size] = labels_batch
      i += 1
      if i * batch_size >= data_size:
          break
  return features, labels

In [ ]:
train_features, train_labels = extract_features_with_pretrained_model(training_dir, 39375)